In [1]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

In [2]:
ds = load_dataset("ailsntua/QEvasion")

df = ds["train"].to_pandas()
df = df.dropna(subset=["clarity_label"])

df["text"] = df["question"] + " " + df["interview_answer"]

X = df["text"]
y = df["clarity_label"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

In [3]:
vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    stop_words="english"
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

In [4]:
model = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    multi_class="auto"
)

model.fit(X_train_vec, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(class_weight='balanced', max_iter=2000, multi_class='auto')

In [5]:
preds = model.predict(X_val_vec)

macro_f1 = f1_score(y_val, preds, average="macro")
print("Macro F1:", round(macro_f1, 4))

print(classification_report(y_val, preds))

Macro F1: 0.5633
                 precision    recall  f1-score   support

     Ambivalent       0.71      0.65      0.68       408
Clear Non-Reply       0.46      0.59      0.52        71
    Clear Reply       0.48      0.51      0.50       211

       accuracy                           0.60       690
      macro avg       0.55      0.58      0.56       690
   weighted avg       0.61      0.60      0.61       690

